In [1]:
import numpy as np
from matplotlib import pyplot as plt
from NeuralNetwork import NeuralNetwork
from utils import get_data, onehot_encoding, grid_search, save_dict_to_file, load_dict_from_file
from Layer import Layer, Input
from functions import accuracy, MSE

In [2]:
best_comb_filename = '/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/output/best_comb7.pkl'
param_grid_filename = '/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/output/param_grid7.pkl'

In [3]:
df = get_data('/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/MONK/monks-3.train')
df_test = get_data('/Users/HP/Desktop/UNI/LM_1/MachineLearning/ML_prj/data/MONK/monks-3.test')

In [4]:
X_train, y_train = df.drop(columns=['target','id']).to_numpy().T, df['target'].apply(lambda x: int(x)).to_numpy().T
X_test, y_test = df_test.drop(columns=['target','id']).to_numpy().T, df_test['target'].apply(lambda x: int(x)).to_numpy().T

# DATA PREPARATION

X_train = onehot_encoding(X_train)
X_test = onehot_encoding(X_test)

In [5]:
best_comb = load_dict_from_file(best_comb_filename)
param_grid = load_dict_from_file(param_grid_filename)

In [6]:
p_best = []
for p_comb in param_grid:
    if p_comb['results']['val_accuracy_mean'] > 90 and p_comb['hidden_act_func'] == 'sigm':
        print(p_comb)
        p_best.append(p_comb)

In [7]:
how_many = 10 # how many elements to print
sorting_key = 'val_accuracy_mean' # value on which the sorting is performed
p_best = []

p_to_sort = [[x, x['results'][sorting_key]] for x in param_grid]
p_best_sorted = sorted(p_to_sort, key=lambda x:-x[1])

for p_comb,loss in p_best_sorted[:how_many]:
    p_best.append(p_comb)
    tmp_dict = p_comb.copy()
    results = tmp_dict.pop('results')
    print(f"val_loss = {results['val_loss_mean']:.2e} +- {results['val_loss_std']:.2e}")
    print(f"val_accuracy = {results['val_accuracy_mean']:.2e} +- {results['val_accuracy_std']:.2e}")
    print(f'hyperparameters : {tmp_dict}')
    print(f'results : {results}')
    print('')

val_loss = 2.59e-01 +- 7.79e-02
val_accuracy = 9.44e+01 +- 3.95e+00
hyperparameters : {'eta': 0.09, 'lam': 3e-05, 'alpha': 0.085, 'epochs': 500, 'n_batch': 'batch', 'scale_eta_batchsize': None, 'dim_hidden': 4, 'hidden_act_func': 'tanh', 'l1_reg': False, 'nest': False, 'loss': 'binary_crossentropy', 'output_act_func': 'sigm', 'elapsed_time': 30.490006685256958}
results : {'train_loss_mean': 0.16626900041344425, 'train_loss_std': 0.022992143332911573, 'val_loss_mean': 0.2588930541930869, 'val_loss_std': 0.07790885718023545, 'train_accuracy_mean': 93.85629251700679, 'train_accuracy_std': 0.8896118539987745, 'val_accuracy_mean': 94.35897435897436, 'val_accuracy_std': 3.9546658500480003, 'train_MSE_mean': 0.04152698539033438, 'train_MSE_std': 0.006148533407712086, 'val_MSE_mean': 0.06381421299759975, 'val_MSE_std': 0.02627955757824755}

val_loss = 2.58e-01 +- 8.41e-02
val_accuracy = 9.43e+01 +- 1.90e+00
hyperparameters : {'eta': 0.075, 'lam': 2e-05, 'alpha': 0.09, 'epochs': 500, 'n_batch':

In [8]:
for i,p_comb in enumerate(p_best):

    best_comb = p_comb.copy()

    results = best_comb.pop('results')
    if best_comb['n_batch'] == 'batch':
        best_comb['n_batch'] = X_train.shape[1]
    elapsed_time = best_comb.pop('elapsed_time')
    if best_comb['scale_eta_batchsize'] == 'lin':
        best_comb['eta'] = best_comb['eta'] * best_comb['n_batch']
    if best_comb['scale_eta_batchsize'] == 'sqrt':
        best_comb['eta'] = best_comb['eta'] * np.sqrt(best_comb['n_batch'])
    best_comb.pop('scale_eta_batchsize')

    #print(best_comb)
    #print(results)
    
    input_layer = Input(X_train.shape[0])
    hidden_layer = Layer(input_layer, best_comb.pop('dim_hidden'), best_comb.pop('hidden_act_func'))
    output_layer = Layer(hidden_layer, 1, 'sigm')

    model = NeuralNetwork(input_layer, output_layer, 'binary_crossentropy', metrics=[accuracy, MSE])
    history = model.retrain(X_train, y_train.reshape((1,X_train.shape[1])), test_data = [X_test,y_test.reshape((1,X_test.shape[1]))], **best_comb)

    '''plt.figure(1)
    plt.plot(history['train_loss'],label='train_loss')
    plt.plot(history['test_loss'], label='test_loss')
    plt.yscale('log')
    plt.xlabel('Epochs', size=15)
    plt.ylabel('Loss functions', size=15)
    plt.title('train_loss vs test_loss', size=18)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.grid()
    plt.legend(fontsize=15)

    plt.figure(2)
    plt.plot(history['train_accuracy'],label='train_accuracy')
    plt.plot(history['test_accuracy'], label='test_accuracy')
    plt.xlabel('Epochs', size=15)
    plt.ylabel('Accuracy', size=15)
    plt.title('train_accuracy vs test_accuracy', size=18)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.grid()
    plt.legend(fontsize=15)'''

    plt.figure(i, figsize=(30,10))
    plt.subplot(1,2,1)
    plt.plot(history['train_MSE'],label='train_MSE')
    plt.plot(history['test_MSE'], label='test_MSE')
    plt.yscale('log')
    plt.xlabel('Epochs', size=15)
    plt.ylabel('Accuracy', size=15)
    plt.title('train_MSE vs test_MSE', size=18)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.grid()
    plt.legend(fontsize=15)

    plt.subplot(1,2,2)
    plt.plot(history['train_accuracy'],label='train_accuracy')
    plt.plot(history['test_accuracy'], label='test_accuracy')
    plt.xlabel('Epochs', size=15)
    plt.ylabel('Accuracy', size=15)
    plt.title('train_accuracy vs test_accuracy', size=18)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.grid()
    plt.legend(fontsize=15)

    plt.show()

TypeError: NeuralNetwork.retrain() got an unexpected keyword argument 'l1_reg'